# 論拠データを整理する
論拠データを作成するためのタスクです。<br>
リサーチフローにおいて論拠データは公開する論文の根拠となるデータとして作成します。<br>
そのため、実験データから論文に必要のないものを除外して公開に適したものになるよう整理する必要があります。<br>
また、論拠データは論文に即したものになるよう、論文の内容に合わせて適宜修正してください。

## 論拠データを整理する
以下の手順に従って論拠データの整理を行ってください。


### 1. リサーチフローのデータをコピーする
親サブフローになっている実験サブフローから実験データをコピーします。<br>
親サブフローを変更する場合はメインメニューに戻り親子関係を変更した後、このタスクを実行してください。<br>
また、リサーチフローに保存していないデータを使用する場合はコピーしたデータの整理を行う際に手動で追加してください。


In [ ]:
# リサーチフローのデータをコピーする

### 2. コピーしたデータを整理する
コピーしたデータのフォルダを表示します。<br>
フォルダ構成や名前の編集、不要なデータの削除などを行い、論拠データを論文に即したものになるよう整理してください。<br>
また、リサーチフローに保存していないデータを使用する場合はここで追加してください。

In [ ]:
# コピーしたデータを整理する

## 論拠データにメタデータを付与する
論拠データのメタデータ登録を行ってください。<br>
セルを実行してメタデータの登録を行った後、このタスクに戻り作業を再開してください。<br>
GakuNin RDMにメタデータを登録する場合はGakuNin RDM上で直接行ってください。<br>

In [ ]:
# メタデータを登録するタスクへアクセスするボタンを表示する

## 再現性を確認する
メタデータの検証と再現性の確認を行ってください。<br>
<br>
※再現性を確認する機能は現在開発中です。


### メタデータを検証する
論拠データのメタデータの検証を行ってください。<br>
セルを実行してメタデータの検証を行った後、このタスクに戻り作業を再開してください。

In [ ]:
# 検証するタスクへアクセスするボタンを表示する

## 収集した文献を整理する
論文執筆に向けて収集した文献を整理し、リサーチフローにアップロードしてください。<br>
また、参考文献は論文に即したものになるよう、論文の内容に合わせて適宜編集してください。

In [ ]:
# 参考文献を保存する

##  GakuNin RDMに保存する
タスクの状態をGakuNin RDMに保存します。

In [ ]:
# GakuNin RDMに保存する
import os

from IPython.core.display import Javascript
from IPython.display import display
import panel as pn

from library.task_director import TaskDirector
from library.utils.save import all_sync_path


script_file_name = 'save_data'
notebook_name = script_file_name+'.ipynb'


class DataSaver(TaskDirector):
    """GRDMに保存するクラスです。

    Attributes:
        instance:
            _abs_root_path (str): 絶対パス
            save_form_box(pn.WidgetBox):フォームを格納する。
            save_msg_output(Message):ユーザーに提示するメッセージを格納する。
    """

    def __init__(self, working_path: str) -> None:
        """DataSaver コンストラクタメソッドです。

        Args:
            working_path (str): 実行Notebookファイルパス
        """
        super().__init__(working_path, notebook_name)

    @TaskDirector.task_cell("1")
    def generate_form_section(self):
        """取得したデータを表示するメソッドです。"""
        # タスク開始によるサブフローステータス管理JSONの更新
        self.doing_task()

        # フォーム定義
        source = all_sync_path(self._abs_root_path)
        self.define_save_form(source)
        # フォーム表示
        pn.extension()
        form_section = pn.WidgetBox()
        form_section.append(self.save_form_box)
        form_section.append(self.save_msg_output)
        display(form_section)
        display(Javascript('IPython.notebook.save_checkpoint();'))


DataSaver(working_path=os.path.abspath('__file__')).generate_form_section()

## サブフローメニューを表示する

サブフローメニューへ遷移するボタンを表示します。

In [ ]:
# サブフローメニューを表示する
import os
from library.task_director import TaskDirector

script_file_name = "make_research_data_management_plan"
notebook_name = script_file_name+'.ipynb'
TaskDirector(os.path.abspath('__file__'), notebook_name).return_subflow_menu()